In [4]:
!bash --login -c "poetry update"

Updating dependencies
Resolving dependencies... (13.9s)

No dependencies to install or update


# Option valuation model based on historical prices

In [1]:
import importlib.util
import os
google_colab_spec = importlib.util.find_spec("google") and importlib.util.find_spec("google.colab")
if google_colab_spec:
    google_colab = importlib.import_module('google.colab')
    google_colab.drive.mount('/content/drive', force_remount=True)
    path = '/content/drive/MyDrive/Colab Notebooks/Ethereum Prices'
else:
    path = f'{os.environ["HOME"]}/Library/CloudStorage/GoogleDrive-chris.perso@gmail.com/My Drive/Colab Notebooks/Ethereum Prices'

import optionspricing
import binanceprices

COUNT_YEARS = 10
BINANCE_DATETIME_FORMAT = "%Y-%m-%d %H-%M-%S"
BINANCE_SYMBOL = 'ETHUSDT'

prices_df = binanceprices.load_prices(path, BINANCE_SYMBOL, COUNT_YEARS)



loading 2014 .no previous data found in /home/python/Library/CloudStorage/GoogleDrive-chris.perso@gmail.com/My Drive/Colab Notebooks/Ethereum Prices/ETHUSDT/2014, loading from binance
.no previous data found in /home/python/Library/CloudStorage/GoogleDrive-chris.perso@gmail.com/My Drive/Colab Notebooks/Ethereum Prices/ETHUSDT/2014, loading from binance
.no previous data found in /home/python/Library/CloudStorage/GoogleDrive-chris.perso@gmail.com/My Drive/Colab Notebooks/Ethereum Prices/ETHUSDT/2014, loading from binance
.no previous data found in /home/python/Library/CloudStorage/GoogleDrive-chris.perso@gmail.com/My Drive/Colab Notebooks/Ethereum Prices/ETHUSDT/2014, loading from binance
.no previous data found in /home/python/Library/CloudStorage/GoogleDrive-chris.perso@gmail.com/My Drive/Colab Notebooks/Ethereum Prices/ETHUSDT/2014, loading from binance
.no previous data found in /home/python/Library/CloudStorage/GoogleDrive-chris.perso@gmail.com/My Drive/Colab Notebooks/Ethereum Pr

## Option valuation model: input parameters here

In [2]:
target_period_hours = 3 * 24
strikes_universe_size = 4
CUT_OFF_YEAR_MONTH = (2022, 1)

instrument_code = BINANCE_SYMBOL[:3]

headers = {"Content-Type": "application/json"}
base_url = "https://www.deribit.com/api/v2/public"

trading_model = optionspricing.TradingModel(base_url, headers, instrument_code, target_period_hours)
trading_model.cutoff_year_month(CUT_OFF_YEAR_MONTH)
target_expiry = trading_model.target_expiry
underlying_price = trading_model.underlying_price
remaining_hours = trading_model.remaining_hours

simulation = trading_model.evaluate(prices_df, strikes_universe_size)
print(simulation)
simulation.allocate([1, 2], 1500. * 2./100.)

target expiry: 2024-02-01 08:00:00+00:00 (62 hours left)
current price: 2304.34
-------------------------------
trading put 2275 and call 2325 (hit ratio: 47%)
cost: 0.019 / value: 0.031, benefit/cost = 1.6x
($) cost: 43.78 / value: 71.78, average gain = 28.00
-------------------------------
trading put 2250 and call 2350 (hit ratio: 42%)
cost: 0.013 / value: 0.024, benefit/cost = 1.9x
($) cost: 28.80 / value: 55.29, average gain = 26.48
-------------------------------
trading put 2225 and call 2375 (hit ratio: 36%)
cost: 0.008 / value: 0.019, benefit/cost = 2.3x
($) cost: 18.43 / value: 42.71, average gain = 24.27
-------------------------------
trading put 2200 and call 2400 (hit ratio: 30%)
cost: 0.005 / value: 0.014, benefit/cost = 2.8x
($) cost: 11.98 / value: 33.06, average gain = 21.08


[{'put': 2250.0, 'call': 2350.0, 'quantity': 1},
 {'put': 2225.0, 'call': 2375.0, 'quantity': 1}]

In [3]:
simulation.options_chain

,value_call,value_call_pct,call_ask,value_put,value_put_pct,put_ask
strike,,,,,,
2200.0,120.656609,NaN,NaN,16.369738,0.007104,0.0023
2225.0,100.067334,NaN,NaN,20.780463,0.009018,0.0036
2250.0,80.818866,NaN,NaN,26.531996,0.011514,0.0055
2275.0,63.561874,NaN,0.0225,34.275003,0.014874,0.0085
2300.0,48.887714,NaN,0.0155,44.600843,0.019355,0.0130
2325.0,37.504339,0.016276,0.0105,58.217469,NaN,0.0190
2350.0,28.754396,0.012478,0.0070,74.467525,NaN,NaN
2375.0,21.926512,0.009515,0.0044,92.639642,NaN,NaN
2400.0,16.692952,0.007244,0.0029,112.406081,NaN,NaN
